Universidade Federal do Rio Grande  
Instituto de Oceanografia  
Programa de Pós-graduação em Oceanologia  
**Disciplina**: Análises de Séries Temporais em Oceanografia – 2023  
**Estudante**: Andrés Eloy Piñango Jauregui (153423)  

# Lista de exercícios 5: Análises Espectrais   
****
**Atividades**:  
Neste exercício, as séries temporais de temperatura, salinidade, concentração de clorofila-a, concentração de matéria orgânica dissolvida e turbidez, da bóia SiMCosta SP01 fornecidas pelo Water Quality Monitoring (WQM), são utilizadas como exemplo para as análises espectrais. As séries possuem 2048 dados horários (para cada variável) e estão disponíveis em http://www.simcosta.furg.br/  

****
### Parte 1: Série de Fourier
#### 1. Visualização dos dados
**a) Antes de iniciar as atividades, verifique o tamanho das séries. Observe a qualidade dos dados, e que eles já foram interpolados**

In [ ]:
### ------------------------------------------------------------------------------------
### Load the libraries to be used in the analysis
### ------------------------------------------------------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import io
from scipy import stats
from scipy import signal

In [ ]:
### ------------------------------------------------------------------------------------
### Download the mat file
### ------------------------------------------------------------------------------------
!wget https://github.com/andresawa/tsa/raw/main/Data/simcosta_sp1_clean2.mat

In [ ]:
### ------------------------------------------------------------------------------------
### Load the mat file
### ------------------------------------------------------------------------------------
data = io.loadmat("./simcosta_sp1_clean2.mat")

In [ ]:
### ------------------------------------------------------------------------------------
### Transform the data into a table
### ------------------------------------------------------------------------------------
simcosta = pd.DataFrame(
    {
        "temp": data["T2"].flatten(),
        "sal": data["S2"].flatten(),
        "chla": data["chla2"].flatten(),
        "turb": data["Turb2"].flatten(),
        "cdom": data["CDOM2"].flatten(),
    }
)

In [ ]:
### ------------------------------------------------------------------------------------
### Transform the time values to actual time data & print the size
### ------------------------------------------------------------------------------------
simcosta["time"] = pd.date_range(
    start="2015-06-29T08:00:00", periods=len(simcosta["temp"]), freq="1H"
)
simcosta = simcosta.set_index("time")
simcosta

**b) Plote as séries na mesma figura (subplot.m)**

In [ ]:
### ------------------------------------------------------------------------------------
### Plot the simcosta data
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(10, 12.5), dpi=300)
ax1 = fig.add_subplot(511)
ax2 = fig.add_subplot(512)
ax3 = fig.add_subplot(513)
ax4 = fig.add_subplot(514)
ax5 = fig.add_subplot(515)
### Subplot 1: temp
simcosta.temp.plot(ax=ax1, color="red")
ax1.grid(linestyle="dashed", alpha=0.3)
ax1.set_title("Temperature SP01", loc="left")
ax1.set_ylabel("°C")
ax1.set_xlabel(None)
ax1.set_xticks([])
### Subplot 2: sal
simcosta.sal.plot(ax=ax2, color="blue")
ax2.grid(linestyle="dashed", alpha=0.3)
ax2.set_title("Salinity SP01", loc="left")
ax2.set_ylabel("psu")
ax2.set_xlabel(None)
ax2.set_xticks([])
### Subplot 3: chla
simcosta.chla.plot(ax=ax3, color="green")
ax3.grid(linestyle="dashed", alpha=0.3)
ax3.set_title("Chlorophyll-a SP01", loc="left")
ax3.set_ylabel("µg $L^{-1}$")
ax3.set_xlabel(None)
ax3.set_xticks([])
### Subplot 4: turb
simcosta.turb.plot(ax=ax4, color="brown")
ax4.grid(linestyle="dashed", alpha=0.3)
ax4.set_title("Turbiduity SP01", loc="left")
ax4.set_ylabel("NTU")
ax4.set_xlabel(None)
ax4.set_xticks([])
### Subplot 2: cdom
simcosta.cdom.plot(ax=ax5, color="#C0DD55")
ax5.grid(linestyle="dashed", alpha=0.3)
ax5.set_title("Coloured dissolved organic matter SP01", loc="left")
ax5.set_ylabel("ppb $QSDE^{-1}$")
ax5.set_xlabel(None)
### Final touch
fig.tight_layout()
plt.show()

#### 2. Análises espectrais com as séries de alta frequência
**a) Utilize um filtro passa-baixa para remover as altas frequências das séries, aplicando a função smoth com span da ordem de 3 dias (i.e. T1 = smooth(T,3*24,’loess’)). Armazene os dados filtrados em novas variáveis**

In [ ]:
### ------------------------------------------------------------------------------------
### Filter (low-pass) the simcosta data
### ------------------------------------------------------------------------------------
# Instead of loess (local regression using weighted linear least squares and a 2nd
# degree polynomial mode), I'm using a Savitzky-Golay filter (a generalized moving
# average with filter coefficients determined by an unweighted linear least-squares
# regression and a 2nd degree polynomial model).
simcosta_lp = pd.DataFrame(
    {
        "temp": signal.savgol_filter(simcosta.temp, 3 * 24, 2),
        "sal": signal.savgol_filter(simcosta.sal, 3 * 24, 2),
        "chla": signal.savgol_filter(simcosta.chla, 3 * 24, 2),
        "turb": signal.savgol_filter(simcosta.turb, 3 * 24, 2),
        "cdom": signal.savgol_filter(simcosta.cdom, 3 * 24, 2),
        "time": simcosta.index,
    }
)
simcosta_lp = simcosta_lp.set_index("time")

**b) Subtraia os dados filtrados das séries originais. Ou seja, a resultante será uma variável com a alta frequência (filtro passa-alta). Armazene o resultado em outra variável**

In [ ]:
### ------------------------------------------------------------------------------------
### Filter (high-pass) the simcosta data
### ------------------------------------------------------------------------------------
simcosta_hp = simcosta - simcosta_lp

**c) Com o banco de dados de alta frequência, faça um estudo de densidade espectral de potência (PSD) para as variáveis temperatura, clorofila e CDOM. Utilize a função g_power.m ou outra de sua preferência. Faça gráficos de PSD x frequência, utilizando plot.m**


In [ ]:
### ------------------------------------------------------------------------------------
### Normalize the high frequency data
### ------------------------------------------------------------------------------------
simcosta_hp_norm = simcosta_hp.apply(stats.zscore)

In [ ]:
### ------------------------------------------------------------------------------------
### Calculate the Power Spectral Density (PSD) for the high frequency data
### ------------------------------------------------------------------------------------
# fs = 24 define day as the unit (1 day have 24 data points)
temp_hp_psd = signal.periodogram(simcosta_hp_norm.temp, fs=24, scaling="density")
chla_hp_psd = signal.periodogram(simcosta_hp_norm.chla, fs=24, scaling="density")
cdom_hp_psd = signal.periodogram(simcosta_hp_norm.cdom, fs=24, scaling="density")

In [ ]:
### ------------------------------------------------------------------------------------
### Plot the Power Spectral Density (PSD) for the high frequency data
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(10, 7.5), dpi=300)
ax1 = fig.add_subplot(311)
ax2 = fig.add_subplot(312)
ax3 = fig.add_subplot(313)
### Subplot 1: Temp
ax1.semilogx(temp_hp_psd[0], temp_hp_psd[1], color="red")
ax1.grid(linestyle="dashed", alpha=0.3)
ax1.set_title("Temperature (high-pass) power spectral density", loc="left")
ax1.set_ylabel("PSD")
ax1.set_ylim(0, 15)
### Subplot 2: Chla
ax2.semilogx(chla_hp_psd[0], chla_hp_psd[1], color="green")
ax2.grid(linestyle="dashed", alpha=0.3)
ax2.set_title("Chlorophyll-a (high-pass) power spectral density", loc="left")
ax2.set_ylabel("PSD")
ax2.set_ylim(0, 15)
### Subplot 3: CDOM
ax3.semilogx(cdom_hp_psd[0], cdom_hp_psd[1], color="#C0DD55")
ax3.grid(linestyle="dashed", alpha=0.3)
ax3.set_title(
    "Colored dissolved organic matter (high-pass) power spectral density", loc="left"
)
ax3.set_xlabel("Frequency (cycles day$^{-1}$)")
ax3.set_ylabel("PSD")
ax3.set_ylim(0, 15)
### Final touch
fig.tight_layout()
plt.show()

**d) Repita o mesmo estudo utilizando os dados de baixa frequência**

In [ ]:
### ------------------------------------------------------------------------------------
### Normalize the low frequency data
### ------------------------------------------------------------------------------------
simcosta_lp_norm = simcosta_lp.apply(stats.zscore)

In [ ]:
### ------------------------------------------------------------------------------------
### Calculate the Power Spectral Density (PSD) for the low frequency data
### ------------------------------------------------------------------------------------
temp_lp_psd = signal.periodogram(simcosta_lp_norm.temp, fs=24, scaling="density")
chla_lp_psd = signal.periodogram(simcosta_lp_norm.chla, fs=24, scaling="density")
cdom_lp_psd = signal.periodogram(simcosta_lp_norm.cdom, fs=24, scaling="density")

In [ ]:
### ------------------------------------------------------------------------------------
### Plot the Power Spectral Density (PSD) for the low frequency data
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(10, 7.5), dpi=300)
ax1 = fig.add_subplot(311)
ax2 = fig.add_subplot(312)
ax3 = fig.add_subplot(313)
### Subplot 1: Temp
ax1.semilogx(temp_lp_psd[0], temp_lp_psd[1], color="red")
ax1.grid(linestyle="dashed", alpha=0.3)
ax1.set_title("Temperature (low-pass) power spectral density", loc="left")
ax1.set_ylabel("PSD")
ax1.set_ylim(0, 30)
### Subplot 2: Chla
ax2.semilogx(chla_lp_psd[0], chla_lp_psd[1], color="green")
ax2.grid(linestyle="dashed", alpha=0.3)
ax2.set_title("Chlorophyll-a (low-pass) power spectral density", loc="left")
ax2.set_ylabel("PSD")
ax2.set_ylim(0, 30)
### Subplot 3: cdom
ax3.semilogx(cdom_lp_psd[0], cdom_lp_psd[1], color="#C0DD55")
ax3.grid(linestyle="dashed", alpha=0.3)
ax3.set_title(
    "Colored dissolved organic matter (low-pass) power spectral density", loc="left"
)
ax3.set_xlabel("Frequency (cycles day$^{-1}$)")
ax3.set_ylabel("PSD")
ax3.set_ylim(0, 30)
### Final touch
fig.tight_layout()
plt.show()

**e) Discuta os resultados**

Ao comparar os periodogramas, torna-se evidente o efeito dos filtros nas frequências. Usando um filtro passa-baixa (Savitzky-Golay), somente aquelas frequências menores que ~0.3 ciclos por dia foram conservadas nos dados de temperatura, clorofila e matéria orgânica dissolvida. Para temperatura e clorofila, a frequência dominante na baixa frequência foi de aproximadamente 0.01 ciclos por dia, o que corresponde a um período de ~100 dias. Para matéria orgânica dissolvida, o pico de máxima frequência na baixa frequência situa-se em ~0.06 ciclos por dia, o que corresponde a um período de aproximadamente 17 dias.  
  
Analisando agora os resultados na alta frequência, os picos dominantes para as três variáveis avaliadas correspondem a 1 ciclo por dia (período de 1 dia). Pode-se observar, para temperatura e clorofila, outro pico em uma frequência de 2 ciclos por dia, que pode ser associado ao ciclo dia-noite.  

****
### Parte II – Análises Espectrais utilizando os métodos de Welch & Lamb  
Para esse exercício, é recomendada a utilização do arquivo de dados freqanal.dat_, que contém uma série temporal de t = 0 a 10 segundos, com medidas a cada 0.005 segundos. A série contém três frequências e algum ruído.  

In [ ]:
### ------------------------------------------------------------------------------------
### Load the freqanual file
### ------------------------------------------------------------------------------------
freqanual = pd.read_table("https://raw.githubusercontent.com/andresawa/tsa/main/Data/freqanual.dat", names=["values"]).squeeze()

#### 3.	Análises Espectrais com o Método de Welch  
**a)	Utilizando a função spectrum.welch.m (ou pwelch.m) e psd.m para fazer uma análise espectral, identifique as frequências dominantes e a relativa potência associada a cada frequência. Teste diferentes segmentos (64, 128, 256). Comente os resultados**

In [ ]:
### ------------------------------------------------------------------------------------
### Calculate the Power Spectral Density (PSD) for the low freqanual data
### ------------------------------------------------------------------------------------
freq_psd64 = signal.welch(freqanual, fs=201, window="hamming", nperseg=64)
freq_psd128 = signal.welch(freqanual, fs=201, window="hamming", nperseg=128)
freq_psd256 = signal.welch(freqanual, fs=201, window="hamming", nperseg=256)

In [ ]:
### ------------------------------------------------------------------------------------
### Plot the Power Spectral Density (PSD) for the low freqanual data
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(10, 10), dpi=300)
ax1 = fig.add_subplot(311)
ax1.semilogy(freq_psd64[0], freq_psd64[1], color="red", label="N = 64")
ax1.semilogy(freq_psd128[0], freq_psd128[1], color="green", label="N = 128")
ax1.semilogy(
    freq_psd256[0], freq_psd256[1], color="blue", label="N = 256"
)
ax1.grid(linestyle="dashed", alpha=0.3)
ax1.set_title("Freqanual power spectral density using Welch’s method", loc="left")
ax1.set_ylabel("PSD")
ax1.set_xlabel("Frequency (Hz)")
ax1.set_ylim(1e-3, 1e1)
ax1.legend()
### Final touch
fig.tight_layout()
plt.show()

No periodograma resultante, três frequências dominantes são claramente identificáveis: a principal em torno de 18Hz, a subsequente aproximadamente em 55Hz e a última em 85Hz. Estas três frequências foram detectadas independentemente do número de dados por segmento. No entanto, a diferença mais notável é a redução do comprimento dos picos à medida que o número de dados nos segmentos aumenta, bem como a emergência de múltiplos picos secundários. Em resumo, ao utilizar segmentos com mais dados no método de Welch, há uma melhoria na resolução, otimizando a identificação dos picos.

**b) Agora adicione à série uma componente de frequência adicional de 110Hz e refaça o item (a). Por exemplo:**
```
T = 0:0.005:10;
y = freqanal’;
f = 110;
y = y+3*sin(2*pi*f*t);
```

In [ ]:
### ------------------------------------------------------------------------------------
### Add the 110Hz frequency to the data
### ------------------------------------------------------------------------------------
freqanual2 = freqanual + (3 * np.sin(2 * np.pi * 110 * np.linspace(0, 10, 2001)))

In [ ]:
### ------------------------------------------------------------------------------------
### Calculate the Power Spectral Density (PSD) for the low freqanual2 data
### ------------------------------------------------------------------------------------
freq2_psd64 = signal.welch(freqanual2, fs=201, window="hamming", nperseg=64)
freq2_psd128 = signal.welch(freqanual2, fs=201, window="hamming", nperseg=128)
freq2_psd256 = signal.welch(freqanual2, fs=201, window="hamming", nperseg=256)

In [ ]:
### ------------------------------------------------------------------------------------
### Plot the Power Spectral Density (PSD) for the low freqanual2 data
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(10, 3.5), dpi=300)
ax1 = fig.add_subplot(111)
ax1.semilogy(freq2_psd64[0], freq2_psd64[1], color="red", label="N = 64")
ax1.semilogy(freq2_psd128[0], freq2_psd128[1], color="green", label="N = 128")
ax1.semilogy(
    freq2_psd256[0], freq2_psd256[1], color="blue", label="N = 256"
)
ax1.grid(linestyle="dashed", alpha=0.3)
ax1.set_title("Freqanual power spectral density using Welch’s method", loc="left")
ax1.set_ylabel("PSD")
ax1.set_xlabel("Frequency (Hz)")
ax1.set_ylim(1e-3, 1e1)
ax1.legend()
### Final touch
fig.tight_layout()
plt.show()

**c) Comente sobre os novos picos de potências. Onde eles aparecem?**

Além dos picos nas frequências descritos anteriormente, apareceu um novo pico em uma frequência de ~90Hz.

### 4. Análises Espectrais com o Método de Lomb

**a) Utilizando os arquivos `tu.dat` (dados independentes - tempo) e `fu.dat` (dados dependentes – variável x) que são dados distribuidos não uniformemente no domínio do tempo. Faça um histograma do logaritmo (base 10) dos intervalos da amostra**


In [ ]:
### ------------------------------------------------------------------------------------
### Load the data
### ------------------------------------------------------------------------------------
time_lomb = pd.read_table("https://github.com/andresawa/tsa/raw/main/Data/tu.dat", names=["time"]).squeeze()
values_lomb = pd.read_table("https://raw.githubusercontent.com/andresawa/tsa/main/Data/fu.dat", names=["values"]).squeeze()

In [ ]:
### ------------------------------------------------------------------------------------
### Plot the histogram
### ------------------------------------------------------------------------------------
diff = pd.Series(np.log10(np.diff(time_lomb)))
fig = plt.figure(figsize=(8, 3.5), dpi=300)
ax1 = fig.add_subplot(111)
diff.hist(ax=ax1)
ax1.set_ylabel("%")
ax1.set_xlabel("Log10(Delta-T)(s)")
ax1.grid(linestyle="dashed", alpha=0.3)
ax1.set_title("Histogram intervals", loc="left")
### Final touch
fig.tight_layout()
plt.show()

**b) Faça o espectro dos dados usando o método de Lomb (lomb.m)**

In [ ]:
### ------------------------------------------------------------------------------------
### Apply the Lomb method
### ------------------------------------------------------------------------------------
frequs = np.linspace(0.01, 1000, 20000)
lomb = signal.lombscargle(time_lomb, values_lomb, frequs, normalize=True)

In [ ]:
### ------------------------------------------------------------------------------------
### Plot the Lomb results
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(10, 3.5), dpi=300)
ax1 = fig.add_subplot(111)
### Subplot 1: Temp
ax1.plot(frequs, lomb, color="red")
ax1.grid(linestyle="dashed", alpha=0.3)
ax1.set_title("Power spectral density using the Lomb-Scargle method", loc="left")
ax1.set_ylabel("Normalized PSD")
ax1.set_xlabel("Frequency (Hz)")
### Final touch
fig.tight_layout()
plt.show()

**c) Identifique as frequências dominantes**

In [ ]:
### ------------------------------------------------------------------------------------
### Find the peaks
### ------------------------------------------------------------------------------------
peaks = signal.find_peaks(lomb, height=0.1)
psd = peaks[1]
freqs = [frequs[n] for n in peaks[0]]
freqs_table = pd.DataFrame({"Frecuency": freqs, "PSD": [*psd.values()][0]}).sort_values(
    by="PSD", ascending=False
)
freqs_table

**d) Comente os resultados**

A frequência dominante, ao aplicar o método de Lomb-Scargle, foi encontrada em ~264Hz, sendo a segunda frequência mais intensa a de 439Hz. Para a normalização, foram usados os valores residuais dos dados em relação a um modelo constante de referência.